In [132]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Optional, List

# === 파일 경로 ===
PARKS_JSON    = Path("서울시 주요 공원현황.json")
POP_CSV       = Path("241106_행정동 단위 서울 생활인구(내국인).csv")  # CSV
DONG_XY_XLSX  = Path("서울시_행정동_좌표.csv")

# === 분석 파라미터 ===
RADII_M = [500, 1000, 2000]                   # 500m / 1km / 2km
HOURS   = [f"{h:02d}" for h in range(7, 23)]  # 07~22시
DATES_TO_INCLUDE: Optional[List[str]] = None

# === 공원 목록 (24개) ===
parks_target_tour = [
    "도산근린공원", "율현공원", "경의선숲길", "문화비축기지", "올림픽공원",
    "송파나루근린공원(석촌호수)", "인왕산도시자연공원(인왕산)", "낙산공원",
    "북한산국립공원(북한산)", "서울로7017", "남산공원", "용산가족공원",
    "효창근린공원(효창공원)",
]
parks_target_han = [
    "뚝섬한강공원","잠원한강공원","여의도한강공원","반포한강공원","망원한강공원",
    "잠실한강공원","양화한강공원","난지한강공원","이촌한강공원","광나루한강공원","강서한강공원"
]

parks_target_add = [
    "길동생태공원", "일자산허브천문공원", "북서울꿈의숲", "방화근린공원", "우장산근린공원(우장산/우장산근린공원)", "서울식물원", 
    "서울대공원", "관악산공원(관악산)", "어린이대공원", "아차산공원(아차산)", "고척근린공원", "푸른수목원", "금천체육공원", "금천폭포근린공원", 
    "수락산도시자연공원(수락산)", "불암산도시자연공원(불암산)", "서울창포원", "용두근린공원(용두공원)", "배봉산근린공원(배봉산)", 
    "국립서울현충원", "보라매공원", "독립공원(서대문독립공원)", "청계산도시자연공원(청계산)", "매헌시민의숲", "서울숲", "달맞이근린공원", 
    "청량근린공원(천장산)", "서서울호수공원", "파리근린공원(파리공원)", "선유도공원", "여의도공원(여의도도시근린공원)", 
    "진관근린공원(구파발폭포/이말산)", "사가정공원(용마산)", "중랑캠핑숲(중랑가족캠핑장)"
]
PARKS_TARGET = parks_target_tour + parks_target_han + parks_target_add


In [133]:
from math import radians, sin, cos, asin, sqrt
import numpy as np
import pandas as pd
from pathlib import Path

def norm_text(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip()
    return s.replace(" ", "")

def haversine_vec(lat1_arr, lon1_arr, lat2, lon2):
    """벡터화 하버사인 거리(m)"""



    R = 6371000.0
    lat1 = np.radians(lat1_arr); lon1 = np.radians(lon1_arr)
    lat2 = np.radians(lat2);     lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# 공원명 별칭(데이터셋 표기 차이를 흡수)
PARK_ALIASES = {
    "송파나루근린공원(석촌호수)": ["석촌호수","송파나루공원"],
    "인왕산도시자연공원(인왕산)": ["인왕산도시자연공원","인왕산"],
    "북한산국립공원(북한산)": ["북한산국립공원","북한산"],
    "서울로 7017": ["서울로7017"],
    "낙산공원": ["낙산공원", "낙산근린공원"],
    "효창근린공원(효창공원)": ["효창공원", "효창근린공원"],
    "훈련원근린공원(훈련원공원)": ["훈련원근린공원"],
    "북악산도시자연공원(북악산)": ["북악산도시자연공원"],
    '삼청근린공원(삼청공원)':["삼청근린공원"], 
    '오금근린공원(오금오름공원/오금공원)':["오금근린공원"], 
    '아시아근린공원(아시아공원)':["아시아근린공원"], 
    '장지근린공원(장지공원)':["장지근린공원"], 
    '도곡근린공원(도곡공원)':["도곡근린공원"], 
    '안산도시자연공원(안산)':["안산도시자연공원"], 
    '백련근린공원(백련산)':["백련근린공원"], 
    '독립공원(서대문독립공원)':["독립공원"], 
    '궁동근린공원(궁동공원)':["궁동근린공원"],
    '성북근린공원(성북공원)':['성북근린공원'], 
    '개운산근린공원(개운산)':["개운산근린공원"], 
    '청량근린공원(천장산)':["청량근린공원"], 
    '오동근린공원(오패산/오동공원)':["오동근린공원"], 
    '수락산도시자연공원(수락산)':["수락산도시자연공원"], 
    '불암산도시자연공원(불암산)':["불암산도시자연공원"], 
    '용두근린공원(용두공원)':["용두근린공원"], 
    '배봉산근린공원(배봉산)':["배봉산근린공원"], 
    '간데메근린공원(간데메공원)':["간데메근린공원"], 
    '답십리근린공원(답십리공원)':["답십리근린공원"], 
    '대현산배수지공원(응봉근린공원)':["대현산배수지공원"], 
    '성수근린공원(성수동구두테마공원)':["성수근린공원"], 
    '중랑캠핑숲(중랑가족캠핑장)':["중랑캠핑숲"], 
    '봉화산근린공원(봉화산)':["봉화산근린공원"], 
    '사가정공원(용마산)':["사가정공원"], 
    '아차산공원(아차산)':["아차산공원"], 
    '봉제산공원(봉제산근린공원/봉제산)':["봉제산공원"], 
    '개화근린공원(개화산)':["개화근린공원"], 
    '염창근린공원(염창산/증미산)':["염창근린공원"], 
    '우장산근린공원(우장산/우장산근린공원)':["우장산근린공원"], 
    '허준공원(허준근린공원)':["허준공원"], 
    '궁산근린공원(궁산)':["궁산근린공원"], 
    '꿩고개근린공원(치현산)':["꿩고개근린공원"], 
    '용왕산근린공원(용왕산)':["용왕산근린공원"], 
    '파리근린공원(파리공원)':["파리근린공원"], 
    '갈산근린공원(갈산공원/갈산)':["갈산근린공원"], 
    '개웅산근린공원(개웅산)':["개웅산근린공원"], 
    '중마루근린공원(중마루공원)':["중마루근린공원"], 
    '여의도공원(여의도도시근린공원)':["여의도공원"], 
    '영등포근린공원(영등포공원)':["영등포근린공원"], 
    '사육신공원(사육신역사공원)':["사육신공원"], 
    '삼일근린공원(삼일공원)':["삼일근린공원"], 
    '관악산공원(관악산)':["관악산공원"], 
    '문화예술공원(서초문화예술공원)':["문화예술공원"], 
    '청계산도시자연공원(청계산)':["청계산도시자연공원"], 
    '인능산도시자연공원(인릉산)':["인능산도시자연공원"], 
    '명일근린공원(명일공원)':["명일근린공원"], 
    '일자산근린공원(일자산)':["일자산근린공원"],
    '진관근린공원(구파발폭포/이말산)':["진관근린공원"],
}

def name_candidates(name: str):
    cs = [name, norm_text(name)]
    for k, alist in PARK_ALIASES.items():
        if name == k or name in alist:
            cs += alist + [norm_text(x) for x in alist]
    return list(dict.fromkeys(cs))  # 중복 제거


In [134]:
import json

# --- 공원 현황 로드 ---
with open(PARKS_JSON, "r", encoding="utf-8") as f:
    parks_json = json.load(f)

# 리스트/딕셔너리 모두 대응
if isinstance(parks_json, list):
    parks = pd.json_normalize(parks_json, max_level=2)
elif isinstance(parks_json, dict):
    list_keys = [k for k, v in parks_json.items() if isinstance(v, list)]
    key = list_keys[0] if list_keys else None
    parks = pd.json_normalize(parks_json[key] if key else parks_json, max_level=2)
else:
    raise ValueError("알 수 없는 공원 JSON 구조")

# 공원명/좌표 컬럼 자동 탐지 + 보정
name_col = next((c for c in parks.columns if "공원" in c or "name" in c.lower() or "p_park" in c.lower()), parks.columns[0])
lat_col  = next((c for c in parks.columns if "lat" in c.lower() or "위도" in c), None)
lng_col  = next((c for c in parks.columns if "lng" in c.lower() or "lon" in c.lower() or "경도" in c), None)
if "p_park" in parks.columns: name_col = "p_park"
if "latitude" in parks.columns: lat_col = "latitude"
if "longitude" in parks.columns: lng_col = "longitude"

parks["_name_raw"] = parks[name_col].astype(str)
parks["_name_norm"] = parks["_name_raw"].map(norm_text)
parks["_lat"] = pd.to_numeric(parks[lat_col], errors="coerce")
parks["_lng"] = pd.to_numeric(parks[lng_col], errors="coerce")
parks_clean = parks.dropna(subset=["_lat","_lng"]).copy()

# --- 타겟 24개 매칭 ---
target_rows, missing = [], []
for target in PARKS_TARGET:
    cands = name_candidates(target)
    sub = parks_clean[parks_clean["_name_raw"].isin(cands) | parks_clean["_name_norm"].isin([norm_text(x) for x in cands])]
    if len(sub) == 0:
        missing.append(target)
    else:
        target_rows.append(sub.iloc[0])

parks_24 = pd.DataFrame(target_rows)
print(f"매칭된 공원 수: {len(parks_24)} / {len(PARKS_TARGET)}")
if missing:
    print("⚠️ 매칭 실패 공원:", missing)

display(parks_24[["_name_raw","_lat","_lng"]].reset_index(drop=True))


매칭된 공원 수: 58 / 58


,_name_raw,_lat,_lng
0,도산근린공원,37.524675,127.035030
1,율현공원,37.472332,127.115594
2,경의선숲길,37.558887,126.925427
3,문화비축기지,37.571718,126.893246
4,올림픽공원,37.520934,127.122959
5,송파나루근린공원(석촌호수),37.506933,127.098300
6,인왕산도시자연공원,37.584900,126.957844
7,낙산근린공원,37.580477,127.007587
8,북한산국립공원,37.624549,127.000938
9,서울로7017,37.556714,126.969575


In [135]:
def read_csv_safely(path: Path) -> pd.DataFrame:
    for enc in ("utf-8-sig", "cp949", "euc-kr", "utf-8"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    raise ValueError("CSV 읽기 실패")

pop_raw = read_csv_safely(POP_CSV)

# 이 CSV 구조: 기준일ID / 시간대구분 / 행정동코드 / 총생활인구수 ...
pop = pop_raw.rename(columns={
    "기준일ID": "stdr_de_id",
    "시간대구분": "tmzon_pd_se",
    "행정동코드": "adstrd_code_se",
})
if "총생활인구수" in pop.columns:
    pop["pop"] = pd.to_numeric(pop["총생활인구수"], errors="coerce")
else:
    protected = {"stdr_de_id", "tmzon_pd_se", "adstrd_code_se"}
    num_cols = [c for c in pop.columns if c not in protected and pd.api.types.is_numeric_dtype(pop[c])]
    pop["pop"] = pop[num_cols].sum(axis=1) if num_cols else np.nan

pop["stdr_de_id"] = pop["stdr_de_id"].astype(str)
pop["tmzon_pd_se"] = pop["tmzon_pd_se"].astype(str).str.zfill(2)
pop["adstrd_code_se"] = pop["adstrd_code_se"].astype(str)
pop["code8"] = pop["adstrd_code_se"].str[:8]

if DATES_TO_INCLUDE:
    pop = pop[pop["stdr_de_id"].isin(DATES_TO_INCLUDE)]
if HOURS:
    pop = pop[pop["tmzon_pd_se"].isin(HOURS)]

pop = pop.dropna(subset=["pop"]).copy()
display(pop.head())



,stdr_de_id,tmzon_pd_se,adstrd_code_se,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,col32,pop,code8
2968,20241106,07,11110515,15612.0677,507.8291,323.5901,1010.0657,359.6889,388.7150,374.1370,...,716.2847,782.7246,688.5992,635.6823,533.4695,318.4346,1002.7530,NaN,15612.0677,11110515
2969,20241106,07,11110530,21268.0888,207.9916,92.4408,218.5917,331.0162,607.0261,788.1320,...,1067.7401,1239.9318,952.2782,778.2612,731.1440,473.0175,1029.1699,NaN,21268.0888,11110530
2970,20241106,07,11110540,5924.1408,62.9667,48.2474,409.8997,350.3734,179.4537,198.8624,...,221.9454,229.2841,171.3394,197.9861,166.5804,136.1212,326.3089,NaN,5924.1408,11110540
2971,20241106,07,11110550,13452.2382,438.8287,340.2616,361.1930,446.3432,376.3729,319.7765,...,571.1793,686.6301,537.4392,568.4102,445.3295,339.3761,932.2709,NaN,13452.2382,11110550
2972,20241106,07,11110560,18248.6093,560.2989,405.2829,384.2804,655.4709,397.3183,349.2393,...,673.2803,796.7349,771.2298,908.3144,850.9032,602.3291,1801.0058,NaN,18248.6093,11110560


In [136]:
# === 좌표 로드 + 표준화 + 조인 (강화/수정 버전) ===
def read_csv_safely(path: Path) -> pd.DataFrame:
    for enc in ("utf-8-sig", "cp949", "euc-kr", "utf-8"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    raise ValueError("CSV 인코딩 자동 판별 실패")

def read_dongxy_any(path: Path) -> pd.DataFrame:
    suf = path.suffix.lower()
    if suf in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    elif suf == ".csv":
        return read_csv_safely(path)
    else:
        raise ValueError("행정동 좌표 파일은 .xlsx/.xls/.csv 중 하나여야 합니다.")

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    def norm(c: str) -> str:
        return str(c).strip().lower().replace(" ", "_")
    return df.rename(columns={c: norm(c) for c in df.columns})

def pick_col(df: pd.DataFrame, candidates: list) -> str:
    for c in candidates:
        if c in df.columns:
            return c
    return ""

# 1) 좌표 파일 읽기
dfxy_raw = read_dongxy_any(DONG_XY_XLSX)
dfxy = normalize_cols(dfxy_raw)

print("[DEBUG] 좌표 원본 컬럼(정규화 후):", list(dfxy.columns))

# 2) 후보군 정의  ← 'gu', 'dong' 추가 (파일 구조: code/sido/gu/dong/lat/lng)
code_cands     = ["code", "adm_cd", "h_code", "adstrd_code_se", "법정동코드", "행정동코드", "emd_cd", "emd_code"]
lat_cands      = ["dong_lat", "lat", "latitude", "위도", "y", "center_lat", "centroid_lat"]
lng_cands      = ["dong_lng", "lng", "lon", "longitude", "경도", "x", "center_lng", "centroid_lng"]
dongname_cands = ["행정동", "행정동명", "법정동", "법정동명", "동", "adm_nm", "adm_dr_nm",
                  "emd_nm", "emd_kor_nm", "dong", "dong_name"]
guname_cands   = ["자치구", "자치구명", "구", "구명", "sgg_nm", "sgg_name",
                  "sig_kor_nm", "시군구명", "sigungu", "gu"]  # ← gu 추가!

# 3) 실제 컬럼 선택
code_col     = pick_col(dfxy, code_cands)
lat_col      = pick_col(dfxy, lat_cands)
lng_col      = pick_col(dfxy, lng_cands)
dongname_col = pick_col(dfxy, dongname_cands)
guname_col   = pick_col(dfxy, guname_cands)

print(f"[DEBUG] 선택된 컬럼 → code:{code_col}, lat:{lat_col}, lng:{lng_col}, 동이름:{dongname_col}, 구이름:{guname_col}")

missing = [name for name, col in {
    "code": code_col, "lat": lat_col, "lng": lng_col,
    "행정동": dongname_col, "자치구": guname_col
}.items() if col == ""]
if missing:
    raise KeyError(f"좌표 파일에서 다음 컬럼을 찾지 못했습니다: {missing}\n"
                   f"현재 사용 가능한 컬럼: {list(dfxy.columns)}")

# 4) 표준 컬럼으로 리네임
dongxy = dfxy.rename(columns={
    code_col:     "code",
    lat_col:      "dong_lat",
    lng_col:      "dong_lng",
    dongname_col: "행정동",
    guname_col:   "자치구",
}).copy()

# 타입/결측 처리
dongxy["code"] = dongxy["code"].astype(str)
dongxy["dong_lat"] = pd.to_numeric(dongxy["dong_lat"], errors="coerce")
dongxy["dong_lng"] = pd.to_numeric(dongxy["dong_lng"], errors="coerce")
dongxy = dongxy.dropna(subset=["dong_lat","dong_lng"]).copy()
dongxy["code8"] = dongxy["code"].str[:8]

print("[DEBUG] 좌표 표준화 후 컬럼:", list(dongxy.columns))

# 5) 조인
need_cols = ["code8","행정동","자치구","dong_lat","dong_lng"]
not_found = [c for c in need_cols if c not in dongxy.columns]
if not_found:
    raise KeyError(f"조인에 필요한 컬럼이 없습니다: {not_found} (현재: {list(dongxy.columns)})")

popxy = pd.merge(
    pop[["stdr_de_id","tmzon_pd_se","code8","pop"]],
    dongxy[need_cols],
    on="code8", how="inner"
)

# 6) 집계
popxy_agg = (popxy
             .groupby(["stdr_de_id","code8","자치구","행정동","dong_lat","dong_lng"], as_index=False)["pop"]
             .sum())

print(f"[OK] popxy_agg: dates={popxy_agg['stdr_de_id'].nunique()} rows={len(popxy_agg)}")
display(popxy_agg.head())


[DEBUG] 좌표 원본 컬럼(정규화 후): ['code', 'sido', 'gu', 'dong', 'lat', 'lng']
[DEBUG] 선택된 컬럼 → code:code, lat:lat, lng:lng, 동이름:dong, 구이름:gu
[DEBUG] 좌표 표준화 후 컬럼: ['code', 'sido', '자치구', '행정동', 'dong_lat', 'dong_lng', 'code8']
[OK] popxy_agg: dates=1 rows=418


,stdr_de_id,code8,자치구,행정동,dong_lat,dong_lng,pop
0,20241106,11110515,종로구,청운효자동,37.584009,126.970626,299188.5444
1,20241106,11110530,종로구,사직동,37.575408,126.965944,569889.0283
2,20241106,11110540,종로구,삼청동,37.590758,126.980996,130928.5339
3,20241106,11110550,종로구,부암동,37.594768,126.965574,216945.6111
4,20241106,11110560,종로구,평창동,37.613029,126.974485,255891.5324


In [137]:
# compute_popweighted_distance 내부에서 사용되는 컬럼명 수정
#   plat = float(p["_lat"])  →  float(p["lat"])
#   plng = float(p["_lng"])  →  float(p["lng"])
#   pname = str(p["_name_raw"]) → str(p["name_raw"])

def compute_popweighted_distance(parks_df, pop_agg_df, radius_m: int, date_col="stdr_de_id"):
    rows = []
    for date_val, grp in pop_agg_df.groupby(date_col):
        dong_lat = grp["dong_lat"].to_numpy()
        dong_lng = grp["dong_lng"].to_numpy()
        dong_pop = grp["pop"].to_numpy()

        for _, p in parks_df.iterrows():
            plat, plng, pname = float(p["lat"]), float(p["lng"]), str(p["name_raw"])
            dists = haversine_vec(dong_lat, dong_lng, plat, plng)
            m = dists <= radius_m
            if m.any() and dong_pop[m].sum() > 0:
                wavg = float((dong_pop[m] * dists[m]).sum() / dong_pop[m].sum())
                total_pop = float(dong_pop[m].sum())
                near_cnt = int(m.sum())
            else:
                wavg, total_pop, near_cnt = np.nan, 0.0, 0

            rows.append({
                "날짜": str(date_val),
                "공원명": pname,
                "반경(m)": radius_m,
                "반경내_행정동수": near_cnt,
                "반경내_생활인구합": total_pop,
                "인구가중_평균거리(m)": wavg,
                "공원_lat": plat,
                "공원_lng": plng
            })
    return pd.DataFrame(rows)


In [138]:
import pandas as pd

def prepare_for_compute(parks_df, pop_df):
    parks = parks_df.copy()
    pop   = pop_df.copy()

    # --- 공원 좌표 통일 + 별칭(lat/lng) ---
    park_lat_col = next((c for c in ["_lat", "latitude", "g_latitude", "lat"] if c in parks.columns), None)
    park_lng_col = next((c for c in ["_lng", "longitude", "g_longitude", "lng", "lon"] if c in parks.columns), None)
    if park_lat_col is None or park_lng_col is None:
        raise KeyError(f"[parks] 위경도 컬럼 없음. columns={list(parks.columns)}")
    parks = parks.rename(columns={park_lat_col: "park_lat", park_lng_col: "park_lng"})
    parks["lat"] = pd.to_numeric(parks["park_lat"], errors="coerce")
    parks["lng"] = pd.to_numeric(parks["park_lng"], errors="coerce")
    parks = parks.dropna(subset=["lat", "lng"])

    # --- 공원명/이름 별칭 (name_raw, name_norm, park_name, 공원명 모두 준비) ---
    name_raw_col  = "_name_raw"  if "_name_raw"  in parks.columns else (
                    "p_park"     if "p_park"     in parks.columns else (
                    "p_name"     if "p_name"     in parks.columns else None))
    name_norm_col = "_name_norm" if "_name_norm" in parks.columns else (
                    "p_park"     if "p_park"     in parks.columns else (
                    "p_name"     if "p_name"     in parks.columns else None))

    # 최소 하나는 있어야 함
    base_name_col = name_raw_col or name_norm_col
    if base_name_col is None:
        raise KeyError(f"[parks] 공원명 컬럼 없음. columns={list(parks.columns)}")

    parks["name_raw"]  = parks[name_raw_col]  if name_raw_col  else parks[base_name_col]
    parks["name_norm"] = parks[name_norm_col] if name_norm_col else parks[base_name_col]
    if "공원명" not in parks.columns:
        parks["공원명"] = parks["name_norm"]
    parks["park_name"] = parks["공원명"]

    # 문자열로 정리
    for c in ["name_raw", "name_norm", "공원명", "park_name"]:
        parks[c] = parks[c].astype(str)

    # --- 행정동/인구 좌표 통일 + 별칭(lat/lng) ---
    if not {"dong_lat", "dong_lng"}.issubset(pop.columns):
        dlat = next((c for c in ["dong_lat","lat","Latitude","latitude","위도","Y","y"] if c in pop.columns), None)
        dlng = next((c for c in ["dong_lng","lng","lon","Longitude","longitude","경도","X","x"] if c in pop.columns), None)
        if dlat is None or dlng is None:
            raise KeyError(f"[pop] 위경도 컬럼 없음. columns={list(pop.columns)}")
        pop = pop.rename(columns={dlat:"dong_lat", dlng:"dong_lng"})

    pop["lat"] = pd.to_numeric(pop["dong_lat"], errors="coerce")
    pop["lng"] = pd.to_numeric(pop["dong_lng"], errors="coerce")
    pop = pop.dropna(subset=["lat", "lng"])

    # (선택) 코드/이름 등의 별칭도 필요시 추가
    if "code8" in pop.columns and "법정동코드" not in pop.columns:
        pop["법정동코드"] = pop["code8"]

    return parks, pop


In [139]:
parks_24_fix, popxy_agg_fix = prepare_for_compute(parks_24, popxy_agg)

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

all_results = []
for r in RADII_M:
    res_r = compute_popweighted_distance(parks_24_fix, popxy_agg_fix, r)
    all_results.append(res_r)
    out_path = OUTPUT_DIR / f"241106_park_popweighted_distance_admin_dong_{r}m.csv"
    res_r.to_csv(out_path, index=False, encoding="utf-8-sig")
    print("저장:", out_path)

result_all = pd.concat(all_results, ignore_index=True)
out_all = OUTPUT_DIR / "241106_park_popweighted_distance_admin_dong_allradii.csv"
result_all.to_csv(out_all, index=False, encoding="utf-8-sig")
print("통합 저장:", out_all)

display(result_all.head(20))


저장: outputs/241106_park_popweighted_distance_admin_dong_500m.csv
저장: outputs/241106_park_popweighted_distance_admin_dong_1000m.csv
저장: outputs/241106_park_popweighted_distance_admin_dong_2000m.csv
통합 저장: outputs/241106_park_popweighted_distance_admin_dong_allradii.csv


,날짜,공원명,반경(m),반경내_행정동수,반경내_생활인구합,인구가중_평균거리(m),공원_lat,공원_lng
0,20241106,도산근린공원,500,0,0.0000,NaN,37.524675,127.035030
1,20241106,율현공원,500,0,0.0000,NaN,37.472332,127.115594
2,20241106,경의선숲길,500,0,0.0000,NaN,37.558887,126.925427
3,20241106,문화비축기지,500,0,0.0000,NaN,37.571718,126.893246
4,20241106,올림픽공원,500,0,0.0000,NaN,37.520934,127.122959
5,20241106,송파나루근린공원(석촌호수),500,0,0.0000,NaN,37.506933,127.098300
6,20241106,인왕산도시자연공원,500,0,0.0000,NaN,37.584900,126.957844
7,20241106,낙산근린공원,500,1,618346.8892,398.924572,37.580477,127.007587
8,20241106,북한산국립공원,500,0,0.0000,NaN,37.624549,127.000938
9,20241106,서울로7017,500,1,146684.6953,371.949523,37.556714,126.969575


In [140]:
# summary = (all_results
#            .groupby(["공원명","반경(m)"], as_index=False)["인구가중_평균거리(m)"]
#            .mean(numeric_only=True)
#            .sort_values(["반경(m)","인구가중_평균거리(m)"]))

# display(summary.head(30))


In [141]:
print("▶ 공원 원본 행수:", len(parks))
print("▶ 좌표 있는 공원 행수:", len(parks_clean))
print("▶ 타겟 24 매칭:", len(parks_24), "/ 58")

if len(parks_24) < 24:
    missing_now = set(PARKS_TARGET) - set(parks_24["_name_raw"])
    print("⚠️ 매칭 실패 공원:", list(missing_now)[:10])

print("\n샘플 공원 5개")
display(parks_24[["_name_raw","_lat","_lng"]].head())

# 좌표 형식/범위 점검
print("\n좌표 NaN 개수:", parks_24["_lat"].isna().sum(), parks_24["_lng"].isna().sum())
print("위도 범위:", parks_24["_lat"].min(), "~", parks_24["_lat"].max())
print("경도 범위:", parks_24["_lng"].min(), "~", parks_24["_lng"].max())


▶ 공원 원본 행수: 151
▶ 좌표 있는 공원 행수: 151
▶ 타겟 24 매칭: 58 / 58

샘플 공원 5개


,_name_raw,_lat,_lng
45,도산근린공원,37.524675,127.035030
128,율현공원,37.472332,127.115594
131,경의선숲길,37.558887,126.925427
126,문화비축기지,37.571718,126.893246
64,올림픽공원,37.520934,127.122959



좌표 NaN 개수: 0 0
위도 범위: 37.4224347 ~ 37.699232
경도 범위: 126.8132285 ~ 127.1547791


In [142]:
# 날짜/시간 값 실제 형태 확인
date_col = "stdr_de_id"
hour_col = "tmzon_pd_se"

print("▶ pop rows:", len(pop))
print("고유 날짜 예시:", pop[date_col].dropna().astype(str).unique()[:5])
print("고유 시간 예시:", pop[hour_col].dropna().astype(str).unique()[:20])

print("\n▶ popxy 조인 전 pop rows:", len(pop))
print("▶ popxy_agg rows:", len(popxy_agg))

if len(popxy_agg) == 0:
    print("⚠️ popxy_agg 비어있음 → 코드/날짜/시간/인구컬럼 의심")

# 코드 길이 점검
print("\n코드 길이(pop code8):", pop["code8"].astype(str).str.len().value_counts().head())
if "code" in dongxy.columns:
    print("코드 길이(dongxy code):", dongxy["code"].astype(str).str.len().value_counts().head())
print("dongxy code8 예시:", dongxy["code8"].astype(str).head().tolist())

# 인구 분포
if "pop" in pop.columns:
    print("\npop 통계:\n", pop["pop"].describe())


▶ pop rows: 6784
고유 날짜 예시: ['20241106']
고유 시간 예시: ['07' '08' '09' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '20'
 '21' '22']

▶ popxy 조인 전 pop rows: 6784
▶ popxy_agg rows: 418

코드 길이(pop code8): code8
8    6784
Name: count, dtype: int64
코드 길이(dongxy code): code
12    891
Name: count, dtype: int64
dongxy code8 예시: ['11110515', '11110530', '11110540', '11110550', '11110560']

pop 통계:
 count      6784.000000
mean      24862.268377
std       17522.594844
min        3313.538900
25%       14347.772025
50%       20234.714450
75%       30019.832950
max      200185.025100
Name: pop, dtype: float64
